I loved simple bank. It was the easies banking interface ever. And it geared to help me save money not spend it, lol.

Sadly, they're gone. So, here's my initial crack at making one.

In [1]:
import pandas as pd
import numpy as np
#import datetime as dt     
today = pd.to_datetime('today').normalize()

In [2]:
# working functions here

def create_date_table(start='2000-01-01', end='2050-12-31'):
    df = pd.DataFrame({"Date": pd.date_range(start, end)})
    df["Day_Name"] = df.Date.dt.day_name()
    # new column
    df["Day"] = df.Date.dt.day
    df["Week"] = df.Date.dt.weekofyear
    df["Quarter"] = df.Date.dt.quarter
    df["Year"] = df.Date.dt.year
    df["Year_half"] = (df.Quarter + 1) // 2
    return df

def calculate_paydays(df_cal, payday = '2021-08-11', frequency = 2, income = 2000):
    #payday = input("When's your next payday? ")
    #frequency = input("How often do you get paid? (enter 1 for weekly, 2 for bi-weekly, 3 for monthy, 4 for twice monthly) ")
    #df_cal['payday'] = np.where(df_cal['Date'] == payday, 'payday', '')
    #income = input("How much money do you get each payday? ")
    start = df_cal.index[df_cal['Date'] == payday].tolist()
    paydays = df_cal[df_cal.index % 14 == start[0]].index
    df_cal['payday'] = np.where(df_cal.index.isin(paydays), 'payday', np.NaN)
    df_cal['income'] = np.where(df_cal.index.isin(paydays), income, np.NaN)
    df_cal['set_aside'] = np.NaN

In [3]:
df_cal = create_date_table('2021-08-01', '2021-12-31')

<ipython-input-2-61fabaf1f02a>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df["Week"] = df.Date.dt.weekofyear


In [4]:
calculate_paydays(df_cal)

In [5]:
goals = pd.read_excel('input_list.xlsx', sheet_name = 'goals')
bills = pd.read_excel('input_list.xlsx', sheet_name = 'bills')

In [ ]:
# what on earth is this function doing....
def set_asides(goals, df_cal):
    goals['save_per_day'] = np.NaN
    for index, row in goals.iterrows():
        due_date = row['due_date']
        paydays_between = df_cal[(df_cal.payday == 'payday')
                                 & (df_cal.Date > today)
                                 & (df_cal.Date < due_date)
                                ].payday.count()
        savings_per_payday = row['amount'] / paydays_between
        #print(savings_per_payday)
        #print(index)
        goals.loc[index,'save_per_day'] = savings_per_payday

    # now take the goals data-frame complete with 
    for index, row in df_cal.iterrows():
        if row['payday'] == 'payday':
            date = row['Date']
            set_aside = goals[['due_date', 'save_per_day']].sum(axis=1).where(goals['due_date'] > date, 0).sum()
            df_cal.loc[index, 'set_aside'] = set_aside
        else: pass

In [8]:
# what on earth is this function doing....
def set_aside_all(goals, bills, df_cal):
    goals['save_per_day'] = np.NaN
    for index, row in goals.iterrows():
        due_date = row['due_date']
        paydays_between = df_cal[(df_cal.payday == 'payday')
                                 & (df_cal.Date > today)
                                 & (df_cal.Date < due_date)
                                ].payday.count()
        savings_per_payday = row['amount'] / paydays_between
        #print(savings_per_payday)
        #print(index)
        goals.loc[index,'save_per_day'] = savings_per_payday

    bills['save_per_day'] = np.NaN
    for index, row in bills.iterrows():
        due_date = row['due_date']
        paydays_between = df_cal[(df_cal.payday == 'payday')
                                 & (df_cal.Date > today)
                                 & (df_cal.Date < due_date)
                                ].payday.count()
        savings_per_payday = row['amount'] / paydays_between
        #print(savings_per_payday)
        #print(index)
        bills.loc[index,'save_per_day'] = savings_per_payday

    # now take the goals data-frame complete with 
    for index, row in df_cal.iterrows():
        if row['payday'] == 'payday':
            date = row['Date']
            set_aside = goals[['due_date'
                               , 'save_per_day']].sum(axis=1).where(goals['due_date'] > date, 0).sum() + bills[['due_date'
                       , 'save_per_day'
                      ]].sum(axis=1).where(bills['due_date'].dt.day > row['Day'], 0).sum()
            df_cal.loc[index, 'set_aside'] = set_aside
        else: pass

In [9]:
set_aside_all(goals, bills, df_cal)

In [10]:
df_cal[df_cal['payday']!='nan']

,Date,Day_Name,Day,Week,Quarter,Year,Year_half,payday,income,set_aside
10,2021-08-11,Wednesday,11,32,3,2021,2,payday,2000.0,545.000000
24,2021-08-25,Wednesday,25,34,3,2021,2,payday,2000.0,280.000000
38,2021-09-08,Wednesday,8,36,3,2021,2,payday,2000.0,415.000000
52,2021-09-22,Wednesday,22,38,3,2021,2,payday,2000.0,150.000000
66,2021-10-06,Wednesday,6,40,4,2021,2,payday,2000.0,415.000000
80,2021-10-20,Wednesday,20,42,4,2021,2,payday,2000.0,150.000000
94,2021-11-03,Wednesday,3,44,4,2021,2,payday,2000.0,415.000000
108,2021-11-17,Wednesday,17,46,4,2021,2,payday,2000.0,33.333333
122,2021-12-01,Wednesday,1,48,4,2021,2,payday,2000.0,298.333333
136,2021-12-15,Wednesday,15,50,4,2021,2,payday,2000.0,33.333333


In [ ]:
# for frequency based payments (monthly) add the bill when the DAY is less than or equal too the payday!

In [ ]:
# re doe the above function to calculate the save per day (really per payday) on the bills
bills['save_per_day'] = np.NaN
for index, row in bills.iterrows():
    due_date = row['due_date']
    paydays_between = df_cal[(df_cal.payday == 'payday')
                             & (df_cal.Date > today)
                             & (df_cal.Date < due_date)
                            ].payday.count()
    savings_per_payday = row['amount'] / paydays_between
    #print(savings_per_payday)
    #print(index)
    bills.loc[index,'save_per_day'] = savings_per_payday

In [ ]:
bills[['due_date'
       , 'save_per_day'
      ]].sum(axis=1).where(bills['due_date'].dt.day > 25, 0)#.sum()

In [7]:
df_cal.head()

,Date,Day_Name,Day,Week,Quarter,Year,Year_half,payday,income,set_aside
0,2021-08-01,Sunday,1,30,3,2021,2,nan,NaN,NaN
1,2021-08-02,Monday,2,31,3,2021,2,nan,NaN,NaN
2,2021-08-03,Tuesday,3,31,3,2021,2,nan,NaN,NaN
3,2021-08-04,Wednesday,4,31,3,2021,2,nan,NaN,NaN
4,2021-08-05,Thursday,5,31,3,2021,2,nan,NaN,NaN
